# Arguments in Smart Contracts
#### 06.5 Writing Smart Contracts
##### Peter Gruber (peter.gruber@usi.ch)
2022-01-12

* Interact with Smart Contracts using arguments

## Setup
See notebook 04.1, the lines below will always automatically load functions in `algo_util.py`, the five accounts and the Purestake credentials

In [ ]:
# Loading shared code and credentials
import sys, os

codepath = '..'+os.path.sep+'..'+os.path.sep+'sharedCode'
sys.path.append(codepath)
from algo_util import *
cred = load_credentials()

# Shortcuts to directly access the 3 main accounts
MyAlgo  = cred['MyAlgo']
Alice   = cred['Alice']
Bob     = cred['Bob']
Charlie = cred['Charlie']
Dina    = cred['Dina']

In [ ]:
from algosdk import account, mnemonic
from algosdk.v2client import algod
from algosdk import transaction
from algosdk.transaction import PaymentTxn
from algosdk.transaction import AssetConfigTxn, AssetTransferTxn, AssetFreezeTxn
from algosdk.transaction import LogicSig

import algosdk.error
import json
import base64
import hashlib

In [ ]:
from pyteal import *

In [ ]:
# Initialize the algod client (Testnet or Mainnet)
algod_client = algod.AlgodClient(algod_token='', algod_address=cred['algod_test'], headers=cred['purestake_token'])

In [ ]:
print(Alice['public'])
print(Bob['public'])
print(Charlie['public'])

#### Quick check of asset holdings, otherwise go to ...
- https://bank.testnet.algorand.network
- https://testnet.algoexplorer.io/dispenser

In [ ]:
asset_holdings_df(algod_client,Alice['public'])

### A few helper functions

In [ ]:
def payment_transaction(creator_mnemonic, amt, rcv, algod_client)->dict:
    params = algod_client.suggested_params()
    add = mnemonic.to_public_key(creator_mnemonic)
    key = mnemonic.to_private_key(creator_mnemonic)
    unsigned_txn = transaction.PaymentTxn(add, params, rcv, amt)
    signed = unsigned_txn.sign(key)
    txid = algod_client.send_transaction(signed)
    return txid

In [ ]:
def lsig_payment_txn(escrowProg, escrow_address, amt, rcv, algod_client):
    params = algod_client.suggested_params()
    unsigned_txn = transaction.PaymentTxn(escrow_address, params, rcv, amt)
    encodedProg = escrowProg.encode()
    program = base64.decodebytes(encodedProg)
    lsig = transaction.LogicSig(program)
    stxn = transaction.LogicSigTransaction(unsigned_txn, lsig)
    txid = algod_client.send_transaction(stxn)
    return txid

In [ ]:
def lsig_payment_txn_arg(escrowProg, escrow_address, amt, rcv, algod_client, arg0):
    params = algod_client.suggested_params()
    unsigned_txn = transaction.PaymentTxn(escrow_address, params, rcv, amt)
    encodedProg = escrowProg.encode()
    program = base64.decodebytes(encodedProg)
    lsig = transaction.LogicSig(program, args=arg0)
    stxn = transaction.LogicSigTransaction(unsigned_txn, lsig)
    txid = algod_client.send_transaction(stxn)
    return txid

In [ ]:
def lsig_payment_txn_note(escrowProg, escrow_address, amt, rcv, algod_client,my_note):
    note = my_note.encode()
    params = algod_client.suggested_params()
    unsigned_txn = transaction.PaymentTxn(escrow_address, params, rcv, amt,None, note)
    encodedProg = escrowProg.encode()
    program = base64.decodebytes(encodedProg)
    lsig = transaction.LogicSig(program)
    stxn = transaction.LogicSigTransaction(unsigned_txn, lsig)
    txid = algod_client.send_transaction(stxn)
    return txid

In [ ]:
def lsig_payment_txn_ASA(escrowProg, escrow_address, amt, rcv, ASA_index, algod_client):
    params = algod_client.suggested_params()
    unsigned_txn = transaction.AssetTransferTxn(escrow_address, params, escrow_address, 0, ASA_index)
    encodedProg = escrowProg.encode()
    program = base64.decodebytes(encodedProg)
    lsig = transaction.LogicSig(program)
    stxn = transaction.LogicSigTransaction(unsigned_txn, lsig)
    txid = algod_client.send_transaction(stxn)
    return txid

## The Cash Machine revisited

#### Remember
* The only way we could interact with Smart Contracts so far was the `note` field of the transaction
* What if we want to send multiple pieces of information (multiple arguments) to a smart contract?

```python
# Condition of the Cash Machine
cashmachine_pyteal = ( 
    Txn.note() == Bytes('{"4711"}') 
    )
```

```python
# Bob had to specify the password in the note
my_note = '{"4711"}'
txn = PaymentTxn(Dispenser['hash'], amt, Bob["public"],my_note.encode())
```


## The Hash lock contract v2
* Use logic sig argument

#### Step 1: Create a hashed version of the password

In [ ]:
secret_password = 'WSC secret'
hash = hashlib.sha256( secret_password.encode() )
print(hash.hexdigest())                                 # The hash is HEX encoded

#### Step 2: Use the hashed password in a different PyTeal condition
* Using arguments for the smart contract (instead of notes)

In [ ]:
hashlock_pyteal = (
    Sha256(Arg(0)) == Bytes(hash.digest())          # <------- Now using the first argument Arg(0)
)

#### Step 3-4: Compile

In [ ]:
# Compile PyTeal -> Teal -> Bytecode for AVM
hashlock_teal = compileTeal(hashlock_pyteal, Mode.Signature, version=3)
print(hashlock_teal)

# Step 3: Compile Teal 

hashlock = algod_client.compile(hashlock_teal)
print("Alice sends these two items to Bob")
print("Smart signature addr: ", hashlock['hash'])
print("Smart signature code: ", hashlock['result'])

#### Step 5: Alice is funding the Smart Signature

In [ ]:
amt = 1001000        # microalgos
txid = payment_transaction(Alice["mnemonic"], amt, hashlock['hash'], algod_client)
print('http://testnet.algoexplorer.io/tx/'+txid)

In [ ]:
pmtx = wait_for_confirmation(algod_client, txid)

#### Step 6: Bob asks the smart signature to sign a transaction with the correct password
* Using arguments for the smart contract (instead of notes)

In [ ]:
# password needs to be encoded
withdrawal_amt = 100000
password       = 'WSC secret'
arg = [password.encode()]                   # arguments need to be a list, even if there is only 1 argument
print(arg)

In [ ]:
txid = lsig_payment_txn_arg(hashlock['result'], hashlock['hash'], withdrawal_amt, Bob['public'], algod_client, arg)
pmtx = wait_for_confirmation(algod_client, txid)

#### Step 7: Check the "Note" field on Algoexplorer
Looks good ... but only at first sight

In [ ]:
print('http://testnet.algoexplorer.io/tx/'+txid)

In [ ]:
# copy the "arguments" from algoexplorer here
arguments =  "V1NDIHNlY3JldA=="
base64.b64decode(arguments)

## Numeric arguments
* Need to encode in Python and decode in PyTeal
* See "Passing parameters using the SDKs" https://developer.algorand.org/docs/get-details/dapps/smart-contracts/frontend/smartsigs/

### Simple quiz: find two numbers that add up to 10
* For example 3+7 = 10

In [ ]:
# Step 1: Conditions as a PyTeal
quiz_pyteal = And(
    Btoi(Arg(0))  + Btoi(Arg(1)) ==  Int(10)
)

# Step 2: Compile PyTeal -> Teal
quiz_teal = compileTeal(quiz_pyteal, Mode.Signature, version=3)
print(quiz_teal)

# Step 3: Compile Teal -> Bytecode for AVM
quiz = algod_client.compile(quiz_teal)
print("Address of smart signature:", quiz['hash'])

**Alice** is funding the smart signature.

In [ ]:
amt = 1001000        # microalgos
txid = payment_transaction(Alice["mnemonic"], amt, quiz['hash'], algod_client)
print('http://testnet.algoexplorer.io/tx/'+txid)

In [ ]:
pmtx = wait_for_confirmation(algod_client, txid)

#### Step 6: Bob asks the smart signature to sign a transaction with the correct password
* Using arguments for the smart contract (instead of notes)

In [ ]:
# password needs to be encoded
withdrawal_amt = 10
# integer parameter
arg0 = (3).to_bytes(8, 'big')
arg1 = (7).to_bytes(8, 'big')

args = [arg0, arg1]                        # arguments need to be a list, even if there is only 1 argument
print(args)

In [ ]:
txid = lsig_payment_txn_arg(quiz['result'], quiz['hash'], withdrawal_amt, Bob['public'], algod_client, args)
pmtx = wait_for_confirmation(algod_client, txid)

#### Now try the following
* Try different arguments that add up to 10
* Try incorrect arguments that do not add up to 10

#### Exercise 1
* Create a quiz with *pythagorean triples*, like, for example, $3^2 + 4^2 = 5^5$
* The quiz pays out 10 micro Algos for every correct solution $a,b,c$ such that $a^2+b^2=c^2$

Solution `Btoi(Arg(0)) * Btoi(Arg(0))  + Btoi(Arg(1))*Btoi(Arg(1)) ==  Btoi(Arg(2))*Btoi(Arg(2))`